# ML Task - Length of Stay (LoS) Prediction

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from dask.distributed import Client

client = Client(
    threads_per_worker=4
)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 2
Total threads: 8,Total memory: 15.05 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:41489,Workers: 2
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 15.05 GiB
Comm: tcp://127.0.0.1:37171,Total threads: 4
Dashboard: http://127.0.0.1:35731/status,Memory: 7.53 GiB
Nanny: tcp://127.0.0.1:45205,


In [3]:
import dask.dataframe as dd
import os

In [4]:
local = True

if local:
    filename = os.path.join('data', 'ICUSTAYS-short.csv')
else:
    filename = 'gs://bigdata2-util/ICUSTAYS.csv'

icu_stays_df = dd.read_csv(filename, parse_dates=['INTIME', 'OUTTIME'], dtype={
    
})

In [5]:
icu_stays_df.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS
0,365,268,110404,280836,carevue,MICU,MICU,52,52,2198-02-14 23:27:38,2198-02-18 05:26:11,3.2490
1,366,269,106296,206613,carevue,MICU,MICU,52,52,2170-11-05 11:05:29,2170-11-08 17:46:57,3.2788
2,367,270,188028,220345,carevue,CCU,CCU,57,57,2128-06-24 15:05:20,2128-06-27 12:32:29,2.8939
3,368,271,173727,249196,carevue,MICU,SICU,52,23,2120-08-07 23:12:42,2120-08-10 00:39:04,2.0600
4,369,272,164716,210407,carevue,CCU,CCU,57,57,2186-12-25 21:08:04,2186-12-27 12:01:13,1.6202


In [6]:
icu_stays_df.describe().compute()

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,FIRST_WARDID,LAST_WARDID,LOS
count,61532.000000,61532.000000,61532.000000,61532.000000,61532.000000,61532.000000,61522.000000
mean,30766.500000,33888.605912,149954.470649,249962.710248,33.085663,32.978206,4.917972
std,17762.902719,28127.690913,28898.895904,28890.574867,19.102204,19.028609,9.638784
min,1.000000,2.000000,100001.000000,200001.000000,7.000000,7.000000,0.000100
25%,15383.750000,12047.500000,124933.750000,224935.500000,14.000000,14.000000,1.108025
50%,30766.500000,24280.500000,149911.500000,249940.000000,33.000000,33.000000,2.092250
75%,46149.250000,54191.500000,174997.250000,274972.500000,52.000000,52.000000,4.483175
max,61532.000000,99999.000000,199999.000000,299999.000000,57.000000,57.000000,173.072500


In [7]:
df_los = icu_stays_df[['HADM_ID', 'INTIME', 'OUTTIME', 'LOS']]

df_los.head(15)

,HADM_ID,INTIME,OUTTIME,LOS
0,110404,2198-02-14 23:27:38,2198-02-18 05:26:11,3.2490
1,106296,2170-11-05 11:05:29,2170-11-08 17:46:57,3.2788
2,188028,2128-06-24 15:05:20,2128-06-27 12:32:29,2.8939
3,173727,2120-08-07 23:12:42,2120-08-10 00:39:04,2.0600
4,164716,2186-12-25 21:08:04,2186-12-27 12:01:13,1.6202
5,158689,2141-04-19 06:12:05,2141-04-20 17:52:11,1.4862
6,130546,2114-06-28 22:28:44,2114-07-07 18:01:16,8.8143
7,129886,2170-10-07 11:28:53,2170-10-14 14:38:07,7.1314
8,135156,2147-11-20 09:02:23,2147-11-21 17:08:52,1.3378
9,171601,2132-10-21 21:11:46,2132-10-22 14:44:48,0.7313


In [8]:
# Statistical information about the Length of Stay (LoS)
df_los[['LOS']].describe().compute()

,LOS
count,61522.000000
mean,4.917972
std,9.638784
min,0.000100
25%,1.108025
50%,2.092250
75%,4.483175
max,173.072500


In [9]:
local = True

if local:
    filename = os.path.join('data', 'EVENTS-short.csv')
else:
    filename = 'gs://bigdata2-util/EVENTS.csv'

events_df = dd.read_csv(filename, parse_dates=['CHARTTIME', 'STORETIME'], dtype={
    'CGID': 'Int64', 'ICUSTAY_ID': 'Int64'
})

In [10]:
# Number of events for each hospital admission

events_count_df = events_df.groupby('HADM_ID')['ROW_ID']\
    .count()\
    .compute()\
    .to_frame()\
    .reset_index()

events_count_df.rename(columns={'ROW_ID': '# of Events'}, inplace=True)

events_count_df.head()

,HADM_ID,# of Events
0,100009,846
1,100020,257
2,100068,1611
3,100182,373
4,100336,1228


In [11]:
clean_dataset = dd.merge(df_los, events_count_df, on='HADM_ID')\
            .compute()

clean_dataset.head()

,HADM_ID,INTIME,OUTTIME,LOS,# of Events
0,125726,2106-04-17 12:26:17,2106-04-18 22:05:39,1.4023,300
1,126219,2107-09-13 22:43:01,2107-09-14 18:34:48,0.8276,146
2,194340,2129-09-03 12:31:31,2129-09-05 23:00:50,2.4370,569
3,169339,2169-05-21 22:04:56,2169-05-25 17:34:03,3.8119,109
4,112077,2167-07-25 18:50:37,2167-07-27 18:37:35,1.9909,396


In [12]:
# Add the SUBJECT_ID column

subjects_df = events_df.groupby('HADM_ID')['SUBJECT_ID']\
    .first()\
    .compute()\
    .to_frame()\
    .reset_index()

subjects_df.head()

,HADM_ID,SUBJECT_ID
0,100009,533
1,100020,9973
2,100068,9889
3,100182,10502
4,100336,8498


In [13]:
clean_dataset = dd.merge(clean_dataset, subjects_df, on='HADM_ID')

# Reorder the columns
clean_dataset = clean_dataset[['HADM_ID', 'SUBJECT_ID', 'INTIME', 'OUTTIME', 'LOS', '# of Events']]
clean_dataset.head() 

,HADM_ID,SUBJECT_ID,INTIME,OUTTIME,LOS,# of Events
0,125726,291,2106-04-17 12:26:17,2106-04-18 22:05:39,1.4023,300
1,126219,291,2107-09-13 22:43:01,2107-09-14 18:34:48,0.8276,146
2,194340,305,2129-09-03 12:31:31,2129-09-05 23:00:50,2.4370,569
3,169339,345,2169-05-21 22:04:56,2169-05-25 17:34:03,3.8119,109
4,112077,85,2167-07-25 18:50:37,2167-07-27 18:37:35,1.9909,396


In [14]:
# Sort the dataset based on INTIME
clean_dataset = clean_dataset.sort_values('INTIME')
clean_dataset.head()

,HADM_ID,SUBJECT_ID,INTIME,OUTTIME,LOS,# of Events
825,156170,6576,2102-06-10 14:00:44,2102-06-13 17:52:40,3.1611,53
209,157537,2712,2103-07-17 17:35:58,2103-07-18 16:15:06,0.9438,26
143,188969,1106,2103-09-30 14:41:06,2103-10-02 14:58:29,2.0121,631
144,188969,1106,2103-10-13 17:30:48,2103-10-13 20:13:16,0.1128,631
145,170024,1106,2103-11-03 13:06:03,2103-11-09 18:37:02,6.2298,2153


In [15]:
# Normalize the events to be in the [0, 1] range

In [16]:
from dask_ml.preprocessing import MinMaxScaler

In [17]:
scaler = MinMaxScaler(feature_range=(0, 1))

clean_dataset['# of Events'] = scaler.fit_transform(clean_dataset[['# of Events']])
clean_dataset.head()

,HADM_ID,SUBJECT_ID,INTIME,OUTTIME,LOS,# of Events
825,156170,6576,2102-06-10 14:00:44,2102-06-13 17:52:40,3.1611,0.000280
209,157537,2712,2103-07-17 17:35:58,2103-07-18 16:15:06,0.9438,0.000086
143,188969,1106,2103-09-30 14:41:06,2103-10-02 14:58:29,2.0121,0.004427
144,188969,1106,2103-10-13 17:30:48,2103-10-13 20:13:16,0.1128,0.004427
145,170024,1106,2103-11-03 13:06:03,2103-11-09 18:37:02,6.2298,0.015348


In [18]:
def preprocess_dataset(events_df, icu_stays_df):
    res_df = icu_stays_df[['HADM_ID', 'INTIME', 'OUTTIME', 'LOS']]

    # Add a column with the number of events for each hospital admission
    events_count_df = events_df.groupby('HADM_ID')['ROW_ID'] \
        .count() \
        .compute() \
        .to_frame() \
        .reset_index()

    events_count_df.rename(columns={'ROW_ID': '# of Events'}, inplace=True)

    res_df = dd.merge(res_df, events_count_df, on='HADM_ID') \
        .compute()

    # Add the SUBJECT_ID column
    subjects_df = events_df.groupby('HADM_ID')['SUBJECT_ID'] \
        .first() \
        .compute() \
        .to_frame() \
        .reset_index()

    res_df = dd.merge(res_df, subjects_df, on='HADM_ID')

    # ReordINer the columns
    res_df = res_df[['HADM_ID', 'SUBJECT_ID', 'INTIME', 'OUTTIME', '# of Events', 'LOS']]
        
    # Sort based on INTIME
    res_df = res_df.sort_values('INTIME')

    # Normalize the '# of Events' column to the [0, 1] range
    scaler = MinMaxScaler(feature_range=(0, 1))
    res_df['# of Events'] = scaler.fit_transform(res_df[['# of Events']])
    
    # Convert the dates to floats
    dt = res_df['INTIME'].values
    res_df['INTIME'] = dt.astype(float)
    
    dt = res_df['OUTTIME'].values
    res_df['OUTTIME'] = dt.astype(float)

    return res_df


dataset_df = preprocess_dataset(events_df, icu_stays_df)
dataset_df

,HADM_ID,SUBJECT_ID,INTIME,OUTTIME,# of Events,LOS
825,156170,6576,4.179391e+18,4.179664e+18,0.000280,3.1611
209,157537,2712,4.214137e+18,4.214219e+18,0.000086,0.9438
143,188969,1106,4.220606e+18,4.220780e+18,0.004427,2.0121
144,188969,1106,4.221740e+18,4.221750e+18,0.004427,0.1128
145,170024,1106,4.223538e+18,4.224077e+18,0.015348,6.2298
...,...,...,...,...,...,...
318,132278,3792,7.472667e+18,7.472711e+18,0.000144,0.5095
390,187638,4843,7.484039e+18,7.484197e+18,0.017558,1.8269
391,187638,4843,7.484685e+18,7.485242e+18,0.017558,6.4406
614,183768,7632,7.523203e+18,7.523675e+18,0.008108,5.4697


## Prediction

In [19]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression

In [20]:
import joblib
import numpy as np

In [21]:
# K-Fold with 10 folds for cr# K-Fold with 10 folds for cross-validatitrain_rowson 
cv = KFold(n_splits=10, random_state=None, shuffle=True)

In [22]:
# Split the data into training and testing data

training_size = int(.8 * len(dataset_df))

training_df = dataset_df.iloc[:training_size, :]
training_df

,HADM_ID,SUBJECT_ID,INTIME,OUTTIME,# of Events,LOS
825,156170,6576,4.179391e+18,4.179664e+18,0.000280,3.1611
209,157537,2712,4.214137e+18,4.214219e+18,0.000086,0.9438
143,188969,1106,4.220606e+18,4.220780e+18,0.004427,2.0121
144,188969,1106,4.221740e+18,4.221750e+18,0.004427,0.1128
145,170024,1106,4.223538e+18,4.224077e+18,0.015348,6.2298
...,...,...,...,...,...,...
219,145893,2369,6.584922e+18,6.585143e+18,0.004571,2.5580
926,141145,9561,6.592910e+18,6.593012e+18,0.001579,1.1725
132,190933,1563,6.605693e+18,6.606383e+18,0.018189,7.9853
699,118007,7695,6.606858e+18,6.607234e+18,0.009120,4.3512


In [23]:
testing_df = dataset_df.iloc[training_size:, :]
testing_df

,HADM_ID,SUBJECT_ID,INTIME,OUTTIME,# of Events,LOS
819,183486,8887,6.634193e+18,6.634485e+18,0.000926,3.3750
700,154714,7695,6.637636e+18,6.638260e+18,0.012506,7.2234
701,189563,7695,6.639470e+18,6.639760e+18,0.006587,3.3556
75,195962,890,6.656425e+18,6.657271e+18,0.039715,9.7871
262,110618,3377,6.659263e+18,6.659890e+18,0.001507,7.2619
...,...,...,...,...,...,...
318,132278,3792,7.472667e+18,7.472711e+18,0.000144,0.5095
390,187638,4843,7.484039e+18,7.484197e+18,0.017558,1.8269
391,187638,4843,7.484685e+18,7.485242e+18,0.017558,6.4406
614,183768,7632,7.523203e+18,7.523675e+18,0.008108,5.4697


In [24]:
# Separate the features (X) from the target variable (y)

X_train_vals = training_df.iloc[:, :-1].values
y_train_vals = training_df.iloc[:, -1].values

In [25]:
X_test_vals = testing_df.iloc[:, :-1].values
y_test_vals = testing_df.iloc[:, -1].values

### Linear Regression

In [26]:
scores = []

with joblib.parallel_backend('dask'):
    for train, test in cv.split(X_train_vals):
        X_train, X_test, y_train, y_test = X_train_vals[train], \
                                           X_train_vals[test], \
                                           y_train_vals[train], \
                                           y_train_vals[test]
        model = LinearRegression().fit(X_train, y_train)
        scores.append(model.score(X_test, y_test))
        
print('Score for each of the folds:', end='\n\n')
for score in scores:
    print(score)
print('\n')

print('Linear Regression Mean Score: {}'.format(np.mean(np.array(scores))), end='\n\n')

print('Score for each feature:', end='\n\n')

for i in range(X_train.shape[-1]):
    print(model.coef_[i])

Score for each of the folds:

0.9999999999690732
0.9999999998419945
0.9999999999464694
0.9999999999382414
0.9999999999261697
0.9999999999826266
0.9999999999547223
0.9999999999732748
0.999999999895581
0.9999999999066201


Linear Regression Mean Score: 0.9999999999334774

Score for each feature:

-9.688274008822447e-11
6.654915352424289e-11
-1.157407432598356e-14
1.1574074324238726e-14
3.121768519967714e-17


### Lasso 

In [27]:
from sklearn.linear_model import Lasso

In [31]:
scores = []

with joblib.parallel_backend('dask'):
    for train, test in cv.split(X_train_vals):
        X_train, X_test, y_train, y_test = X_train_vals[train], X_train_vals[test], y_train_vals[train], y_train_vals[test]
        model = Lasso(alpha=0.01).fit(X_train, y_train)
        scores.append(model.score(X_test, y_test))
        
print('Score for each of the folds:', end='\n\n')
for score in scores:
    print(score)
print('\n')

print('Lasso Regression Mean Score: {}'.format(np.mean(np.array(scores))), end='\n\n')

print('Score for each feature:', end='\n\n')

for i in range(X_train.shape[-1]):
    print(model.coef_[i])

Score for each of the folds:

0.0761763636588918
0.11998321699855607
0.17002554907279732
0.10681892300584284
-0.2033484914630248
-13.514850675278563
0.11950751504359758
0.049614007261668425
0.15861211935532504
0.13074134291373274


Lasso Regression Mean Score: -1.2786720129431175

Score for each feature:

2.347184469321773e-06
-8.15974652515346e-06
-4.398196761562795e-18
4.351493651961214e-18
31.509863656350696


### Random Forest Regressor

In [29]:
from sklearn.ensemble import RandomForestRegressor

In [30]:
scores = []

with joblib.parallel_backend('dask'):
    for train, test in cv.split(X_train_vals):
        X_train, X_test, y_train, y_test = X_train_vals[train], X_train_vals[test], y_train_vals[train], y_train_vals[test]
        model = RandomForestRegressor().fit(X_train, y_train)
        scores.append(model.score(X_test, y_test))
        
print('Score for each of the folds:', end='\n\n')
for score in scores:
    print(score)

print('\n')

print('Random Forest Regressor Mean Score: {}'.format(np.mean(np.array(scores))), end='\n\n')

Score for each of the folds:

0.5627101299228862
-0.30798707348304166
0.35424001316516474
0.606410440338067
0.6840641144165259
0.4390133656174553
0.16321596007639994
0.19937032258041965
0.84073536176418
0.4590333964420684


Random Forest Regressor Mean Score: 0.4000806030840126

